In [10]:
import requests
import csv 
from bs4 import BeautifulSoup
import lxml
import re
from tqdm import tqdm
import time

import pandas as pd

In [11]:
list_url= 'https://www.goodreads.com/list/show/134.Best_Non_Fiction_no_biographies_'

books_df = pd.DataFrame(columns = ['Title', 'Author', 'Description' 
                                   ,'Num_ratings', 'Avg_rating', 'Genres_list',
                                   'Main_genre', 'Secondary_genre' , 'Num_pages', 'ISBN','Link'])


In [12]:
def get_book_list(list_url, num_pages = 10):
    
    books_dict = {}
    
    for i in tqdm(range(1,num_pages)):
        bookpage=str(i)
        stuff=requests.get(list_url + '?page=' + bookpage)
        soup = BeautifulSoup(stuff.text, 'html.parser')

        for e in soup.select('.bookTitle'):
            books_dict[e.get_text(strip = 'True')] = e['href']

        time.sleep(2)
    
    return books_dict

In [13]:
def get_key(val, my_dict): 
    for key, value in my_dict.items(): 
         if val == value: 
                return key 
  
    return "key doesn't exist"

In [14]:
def get_main_genres(d):
    
    genre_dict = {}
    genres = d.select("div.elementList div.right a")
        
    for g in genres:
        genre_dict[g['href'].partition('=')[2]] = int(re.sub(',','',g.get_text().split()[0]))
        
    if len(genre_dict)>0:
        Max_genre = max(genre_dict, key= lambda x: genre_dict[x]) 
    else:
        Max_genre = None
    
    if len(genre_dict)>1:
        second_largest_value = list(sorted(genre_dict.values()))[-2]
    else:
        second_largest_value = None
    
    second_genre = get_key(second_largest_value, genre_dict)
        
    return genre_dict, Max_genre, second_genre

In [15]:
def get_book_details(book_url):
    
    book_dict = {}
    
    page=requests.get(book_url)
    soup=BeautifulSoup(page.text, 'html.parser')
    
    title = soup.select("meta[property='og:title']")[0]['content']
    author = soup.find_all('a', class_ = 'authorName')[0].get_text(strip = True)

    desc = soup.find('div', id = 'description')
    if desc is not None:
        try:
            description = desc.find_all('span')[1].get_text()
        except:
            description = desc.find_all('span')[0].get_text()
    else:
        description = None

    if soup.find('meta', itemprop = 'ratingCount') is not None:
        num_ratings = soup.find('meta', itemprop = 'ratingCount')['content']
    else:
        num_ratings = None

    if soup.find('span', itemprop = 'ratingValue') is not None:    
        rating = soup.find('span', itemprop = 'ratingValue').get_text(strip = 'True')
    else:
        rating = None

    genre_list, main_genre, secondary_genre = get_main_genres(soup)

    if soup.select("meta[property='books:page_count']") is not None and len(soup.select("meta[property='books:page_count']")) != 0: 
        num_pages = soup.select("meta[property='books:page_count']")[0]['content']
    else:
        num_pages = None

    if soup.select("meta[property='books:isbn']") is not None and len(soup.select("meta[property='books:isbn']")) != 0:
        isbn = soup.select("meta[property='books:isbn']")[0]['content']
    else:
        isbn = None

    book_dict = {
        'Title' : title,
        'Author' : author,
        'Description' : description,
        'Num_ratings' : num_ratings,
        'Avg_rating' : rating,
        'Genres_list' : genre_list,
        'Main_genre' : main_genre,
        'Secondary_genre': secondary_genre,
        'Num_pages' : num_pages,
        'ISBN' : isbn,
        'Link' : book_url
    }
    
    return book_dict

In [16]:
books_dict = {}

books_dict = get_book_list(list_url, num_pages = 3)

100%|██████████| 2/2 [00:16<00:00,  8.05s/it]


In [17]:
for book in tqdm(list(books_dict.values())):
    
    book_url = 'https://www.goodreads.com/' + str(book)
    print("Extracting from {}".format(book_url))
    books_df = books_df.append(get_book_details(book_url), ignore_index= True)

  0%|          | 0/200 [00:00<?, ?it/s]

Extracting from https://www.goodreads.com//book/show/21996.The_Devil_in_the_White_City


  0%|          | 1/200 [00:05<18:53,  5.69s/it]

Extracting from https://www.goodreads.com//book/show/1202.Freakonomics


  1%|          | 2/200 [00:11<18:39,  5.65s/it]

...

 44%|████▍     | 89/200 [07:15<10:13,  5.53s/it]

Extracting from https://www.goodreads.com//book/show/7940583-chasing-the-devil


 45%|████▌     | 90/200 [07:23<11:39,  6.36s/it]

Extracting from https://www.goodreads.com//book/show/16884.The_Making_of_the_Atomic_Bomb


KeyboardInterrupt: 

In [290]:
books_df

,Title,Author,Description,Num_ratings,Avg_rating,Genres_list,Main_genre,Secondary_genre,Num_pages,ISBN,Link
0,Diplomacy,Henry Kissinger,"A brilliant, sweeping history of diplomacy tha...",6452,4.22,"{'history': 391, 'politics': 325, 'non-fiction...",history,politics,912,9780671510992,https://www.goodreads.com//book/show/781183.Di...
1,The Happiness Advantage,Shawn Achor,Our most commonly held formula for success is ...,23766,4.18,"{'non-fiction': 555, 'psychology': 517, 'self-...",non-fiction,psychology,272,9780307591562,https://www.goodreads.com//book/show/9484114-t...
2,The Stuff of Thought,Steven Pinker,New York Times bestselling author Steven Pinke...,9712,3.90,"{'non-fiction': 593, 'science': 412, 'psycholo...",non-fiction,science,499,9780670063277,https://www.goodreads.com//book/show/373969.Th...
3,Civilization and Its Discontents,Sigmund Freud,It stands as a brilliant summary of the views ...,30963,3.79,"{'psychology': 1218, 'philosophy': 909, 'non-f...",psychology,philosophy,127,9780393301588,https://www.goodreads.com//book/show/357636.Ci...
4,The Corporation,Joel Bakan,The inspiration for the film that won the 2004...,4084,4.07,"{'non-fiction': 196, 'economics': 119, 'politi...",non-fiction,economics,228,9780743247467,https://www.goodreads.com//book/show/108583.Th...
...,...,...,...,...,...,...,...,...,...,...,...
306,A Treasury of Deception,Michael Farquhar,"We may say that honesty is the best policy, bu...",657,3.62,"{'history': 84, 'non-fiction': 62, 'historical...",history,non-fiction,304,9780143035442,https://www.goodreads.com//book/show/401503.A_...
307,The Girls Who Went Away,Ann Fessler,A powerful and groundbreaking revelation of th...,5819,4.19,"{'non-fiction': 438, 'history': 279, 'feminism...",non-fiction,history,354,9781594200946,https://www.goodreads.com//book/show/116477.Th...
308,Fatal Vision,Joe McGinniss,Fatal Vision is the electrifying true story of...,22182,4.11,"{'true-crime': 739, 'non-fiction': 306, 'crime...",true-crime,non-fiction,684,9780451165664,https://www.goodreads.com//book/show/333907.Fa...
309,The Divided Self,R.D. Laing,"In The Divided Self (1960), Laing contrasted t...",4869,4.09,"{'psychology': 463, 'non-fiction': 182, 'philo...",psychology,non-fiction,224,9780140135374,https://www.goodreads.com//book/show/496585.Th...


In [292]:
books_df.to_csv('scraped_books.csv')